# TFG - Anàlisi de l’abandonament escolar prematur a Catalunya
## Tractament de Dades Any 2022
Jahziel Hidalgo Artigas



## Taula de continguts<a class="anchor" id="index"></a>
* [Imports](#imports)
* [Funcions](#funcions)
* [Càrrega Datasets](#datasets)
* [Comarques](#comarques)
* [Dades Formació](#formacio)
    * [Anàlisi per Gènere](#formacio_genere)
    * [Formació d'Adults](#formacio_adults)
* [Dades Població](#poblacio)
   * [Població per Gènere](#poblacio_genere)
   * [Població per Edat](#poblacio_edat)
   * [Població Activa (16-24 anys)](#poblacio_activa)
   * [Població Inactiva](#poblacio_inactiva)
   * [Atur per Edat](#atur)
   * [Atur per Gènere](#atur_genere)
   * [Percentatge/Ratio Estudiants](#pct_estudiants)
   * [Població per Nacionalitat](#poblacio_nacionalitat)
* [Dades Socioeconòmiques](#dades_socioeconomiques)
    * [Repetidors](#repetidors)
    * [Total Repetidors per Comarca](#repetidors_comarca)
    * [Impacte de la Nacionalitat](#extrangers)


### Imports<a class="anchor" id="imports"></a>

In [177]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import os
from unidecode import unidecode

### Funcions<a class="anchor" id="funcions"></a>

In [180]:
def clean_dataframe(df):
    # Netejar les columnes: treure els accents, substituir els espais per guions baixos i convertir a minúscules
    df.columns = [unidecode(col.replace(' ', '_').lower()) for col in df.columns]
    
    # Netejar les dades: treure els accents, convertir a minúscules i processar només les cadenes de text
    df = df.apply(lambda x: x.apply(lambda val: unidecode(str(val)).lower() if isinstance(val, str) else val) 
                  if x.dtype == 'object' else x, axis=0)
    
    return df

def filtrar_comarques(df, columna):
    comarques_valides = [
        'alt camp', 'alt emporda', 'alt penedes', 'alt urgell', 'alta ribagorca', 
        'anoia', 'aran', 'bages', 'baix camp', 'baix ebre', 'baix emporda', 
        'baix llobregat', 'baix penedes', 'barcelones', 'bergueda', 'cerdanya', 
        'conca de barbera', 'garraf', 'garrigues', 'garrotxa', 'girones', 'maresme', 
        'moianes', 'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira', 
        "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles', 
        'segarra', 'segria', 'selva', 'solsones', 'tarragones', 'terra alta', 'urgell', 
        'valles occidental', 'valles oriental'
    ]
    
    # Filtrar les files que continguin valors en la columna dins de la llista de comarques vàlides
    df = df[df[columna].isin(comarques_valides)].reset_index(drop=True)
    
    return df


### Càrrega Datasets<a class="anchor" id="datasets"></a>

In [183]:
df_comarques = pd.read_csv('csv/2022/comarques.csv', delimiter=';', skiprows=3)
df_formacio_sexe = pd.read_csv('csv/2022/nivell_formacio_sexe_2022.csv', delimiter=';') 
df_formacio_adults = pd.read_csv('csv/2022/formacio_adults_2022.csv', delimiter=';', skiprows=5)
df_poblacio_genere = pd.read_csv('csv/2022/poblacio.csv', delimiter=';')
df_poblacio_edat = pd.read_csv('csv/2022/poblacio_edat.csv', delimiter=';')
df_poblacio_activa = pd.read_csv('csv/2022/poblacio_activa_2022.csv', delimiter=';', skiprows=6)
df_poblacio_inactiva = pd.read_csv('csv/2022/poblacio_inactiva_2022.csv', delimiter=';', skiprows=5)
df_poblacio_nacionalitat = pd.read_csv('csv/2022/poblacio_nacionalitat_2022.csv', delimiter=';', skiprows=7)
df_repetidors = pd.read_csv('csv/2022/alumnes_repetidors_2022.csv', delimiter=';', encoding='latin1')
df_atur_edat = pd.read_csv('csv/2022/atur_edat_2022.csv', delimiter=';', skiprows=7)
df_atur_genere = pd.read_csv('csv/2022/atur_genere_2022.csv', delimiter=';', skiprows=6)

In [8]:
datasets = {
    'Comarques': df_comarques,
    'Formació Sexe': df_formacio_sexe,
    'Formació Adults': df_formacio_adults,
    'Població Sexe': df_poblacio_genere,
    'Població Edat': df_poblacio_edat,
    'Població Activa': df_poblacio_activa,
    'Població Inactiva': df_poblacio_inactiva,
    'Població Nacionalitat': df_poblacio_nacionalitat,
    'Repetidors': df_repetidors,
}

# Iterar sobre el diccionari per mostrar les primeres files de cada dataset
for name, df in datasets.items():
    print(f"Primeres files de {name}:")
    display(df.head()) 
    print("\n") 

Primeres files de Comarques:


,Nivell,Codi,Nom
0,Comarca,1,Alt Camp
1,Municipi,430017,Aiguamúrcia
2,Municipi,430056,Alcover
3,Municipi,430108,Alió
4,Municipi,430347,Bràfim




Primeres files de Formació Sexe:


,any,comarca o Aran,sexe,nivell de formació assolit,concepte,estat,valor
0,2022,Alt Camp,homes,analfabetisme,població de 15 anys o més,NaN,27
1,2022,Alt Camp,homes,educació primària incompleta,població de 15 anys o més,NaN,501
2,2022,Alt Camp,homes,educació primària,població de 15 anys o més,NaN,1944
3,2022,Alt Camp,homes,primera etapa d’educació secundària i similar,població de 15 anys o més,NaN,7527
4,2022,Alt Camp,homes,"segona etapa d’educació secundària, amb orient...",població de 15 anys o més,NaN,1974




Primeres files de Formació Adults:


...

,Unnamed: 0,"Població pensionista de jubilació, prejubilació",Població pensionista d'invalidesa,Població estudiant,Població amb una altra situació d'inactivitat,Total població inactiva
0,Alt Camp,7558,1094,2090,5614,16356
1,Alt Empordà,20492,2163,5896,27505,56056
2,Alt Penedès,16105,2289,4900,14457,37751
3,Alt Urgell,3528,477,893,4222,9120
4,Alta Ribagorça,592,102,195,569,1458




Primeres files de Població Nacionalitat:


,Unnamed: 0,Espanyola,Resta UE,Resta d'Europa,Àfrica,Amèrica del Nord i Central,Amèrica del Sud,Àsia i Oceania,Total
0,Alt Camp,39463,1320,491,2884,240,741,330,45469
1,Alt Empordà,108698,10998,2780,13567,2558,4222,1127,143950
2,Alt Penedès,98944,1801,792,5725,693,2022,833,110810
3,Alt Urgell,17783,944,301,457,182,668,123,20458
4,Alta Ribagorça,3473,179,62,140,37,89,9,3989




Primeres files de Repetidors:


,Curs,Estudi,Codi àrea territorial,Àrea territorial,Codi comarca,Comarca,Codi municipi,Municipi,Codi districte,Naturalesa,Titularitat,Nivell,Sexe,Concert,Resultat de l'avaluació,Alumnes avaluats
0,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,1,DONA,SI,PROMOCIONEN SENSE PENDENTS,17
1,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,1,HOME,SI,PROMOCIONEN AMB PENDENTS,5
2,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,1,HOME,SI,PROMOCIONEN SENSE PENDENTS,38
3,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,2,DONA,SI,NO PROMOCIONEN,1
4,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,2,DONA,SI,PROMOCIONEN AMB PENDENTS,6


### Comarques<a class="anchor" id="comarques"></a>

L'estudi es farà per comarques, així que necessitem obtenir el codi de les comarques, el qual ens servirà per agrupar el conjunt final.

In [11]:
# Primerament, netegem el dataset per assegurar-nos que no hi hagi problemes amb els noms de les columnes, accents, etc.
df_comarques = clean_dataframe(df_comarques)

In [12]:
df_comarques.head()

,nivell,codi,nom
0,comarca,1,alt camp
1,municipi,430017,aiguamurcia
2,municipi,430056,alcover
3,municipi,430108,alio
4,municipi,430347,brafim


In [13]:
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nivell  990 non-null    object
 1   codi    990 non-null    int64 
 2   nom     990 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [14]:
# Filtrarem les dades per seleccionar només aquelles files on el 'nivell' sigui 'comarca',
# i només seleccionarem les columnes 'codi' i 'nom' que seran necessàries per a l'anàlisi posterior.
df_comarques = df_comarques[df_comarques['nivell'] == 'comarca'][['codi', 'nom']].reset_index(drop=True)
# Convertim la columna 'codi' a format de dos dígits
df_comarques['codi'] = df_comarques['codi'].astype(str).str.zfill(2)
df_comarques.head()

,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca


In [15]:
df_comarques

,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca
5,06,anoia
6,07,bages
7,08,baix camp
8,09,baix ebre
9,10,baix emporda


In [16]:
# Afegim manualment el codi d'Aran (39)
df_comarques.loc[len(df_comarques)] = ['39', 'aran']
df_comarques = df_comarques.sort_values(by='codi').reset_index(drop=True)
df_comarques.tail()

,codi,nom
38,39,aran
39,40,valles occidental
40,41,valles oriental
41,42,moianes
42,43,llucanes


In [17]:
df_comarques['codi'] = df_comarques['codi'].astype(str)
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   codi    43 non-null     object
 1   nom     43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [18]:
# Comprovem els noms únics de les comarques per assegurar-nos que no hi ha comarques duplicades
df_comarques['nom'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'bages', 'baix camp', 'baix ebre',
       'baix emporda', 'baix llobregat', 'baix penedes', 'barcelones',
       'bergueda', 'cerdanya', 'conca de barbera', 'garraf', 'garrigues',
       'garrotxa', 'girones', 'maresme', 'montsia', 'noguera', 'osona',
       'pallars jussa', 'pallars sobira', "pla d'urgell",
       "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles',
       'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'aran', 'valles occidental',
       'valles oriental', 'moianes', 'llucanes'], dtype=object)

In [19]:
# Imprimim el nombre de comarques úniques que tenim en el dataset
print(f"Hi han {len(df_comarques['nom'].unique())} comarques")

Hi han 43 comarques


In [20]:
# Crear el diccionari de codis (comarca -> codi)
comarques_dict = df_comarques.set_index('nom')['codi'].to_dict()
comarques_dict

{'alt camp': '01',
 'alt emporda': '02',
 'alt penedes': '03',
 'alt urgell': '04',
 'alta ribagorca': '05',
 'anoia': '06',
 'bages': '07',
 'baix camp': '08',
 'baix ebre': '09',
 'baix emporda': '10',
 'baix llobregat': '11',
 'baix penedes': '12',
 'barcelones': '13',
 'bergueda': '14',
 'cerdanya': '15',
 'conca de barbera': '16',
 'garraf': '17',
 'garrigues': '18',
 'garrotxa': '19',
 'girones': '20',
 'maresme': '21',
 'montsia': '22',
 'noguera': '23',
 'osona': '24',
 'pallars jussa': '25',
 'pallars sobira': '26',
 "pla d'urgell": '27',
 "pla de l'estany": '28',
 'priorat': '29',
 "ribera d'ebre": '30',
 'ripolles': '31',
 'segarra': '32',
 'segria': '33',
 'selva': '34',
 'solsones': '35',
 'tarragones': '36',
 'terra alta': '37',
 'urgell': '38',
 'aran': '39',
 'valles occidental': '40',
 'valles oriental': '41',
 'moianes': '42',
 'llucanes': '43'}

### Dades Formació<a class="anchor" id="formacio"></a>

#### Anàlisi per Gènere<a class="anchor" id="formacio_genere"></a>

In [ ]:
df_formacio_sexe = pd.read_csv('csv/2021/nivell_formacio_sexe_2021.csv', delimiter=';') 

In [23]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_genere_clean = clean_dataframe(df_formacio_sexe)
df_formacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1548 non-null   int64  
 1   comarca_o_aran              1548 non-null   object 
 2   sexe                        1548 non-null   object 
 3   nivell_de_formacio_assolit  1548 non-null   object 
 4   concepte                    1548 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1548 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 84.8+ KB


In [24]:
# Corregir nom columna comarca i filtrar per 'total' i només per l'any 2022
df_formacio_genere_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['nivell_de_formacio_assolit'] != 'total']
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['any'] == 2022]

In [25]:
# Filtrar per comarques
df_formacio_genere_clean = filtrar_comarques(df_formacio_genere_clean, 'comarca')

In [26]:
# Comprovem els canvis realitzats
df_formacio_genere_clean.info()
display(df_formacio_genere_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1386 non-null   int64  
 1   comarca                     1386 non-null   object 
 2   sexe                        1386 non-null   object 
 3   nivell_de_formacio_assolit  1386 non-null   object 
 4   concepte                    1386 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1386 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 75.9+ KB


,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor
0,2022,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27
1,2022,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,501
2,2022,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,1944
3,2022,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7527
4,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1974
5,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,2454
6,2022,alt camp,homes,ensenyaments de grau superior de formacio prof...,poblacio de 15 anys o mes,NaN,2325
7,2022,alt camp,homes,graus universitaris de fins a 240 credits i si...,poblacio de 15 anys o mes,NaN,1350
8,2022,alt camp,homes,graus universitaris de mes de 240 credits i si...,poblacio de 15 anys o mes,NaN,1080
9,2022,alt camp,homes,masters universitaris i similars,poblacio de 15 anys o mes,NaN,366


In [27]:
df_formacio_genere_clean['nivell_de_formacio_assolit'].unique()

array(['analfabetisme', 'educacio primaria incompleta',
       'educacio primaria',
       "primera etapa d'educacio secundaria i similar",
       "segona etapa d'educacio secundaria, amb orientacio general",
       "segona etapa d'educacio secundaria, amb orientacio professional",
       'ensenyaments de grau superior de formacio professional i similars',
       'graus universitaris de fins a 240 credits i similars',
       'graus universitaris de mes de 240 credits i similars',
       'masters universitaris i similars', 'doctorat universitari'],
      dtype=object)

In [29]:
# Creem el mapeig de categories educatives
categoria_map = {
    'analfabetisme': 'analfabetisme',
    'educacio primaria incompleta': 'educacio primaria incompleta',
    'educacio primaria': 'educacio primaria',
    'primera etapa d\'educacio secundaria i similar': 'educacio secundaria',
    'segona etapa d\'educacio secundaria, amb orientacio general': 'educacio secundaria',
    'segona etapa d\'educacio secundaria, amb orientacio professional': 'educacio secundaria',
    'ensenyaments de grau superior de formacio professional i similars': 'formacio professional',
    'graus universitaris de fins a 240 credits i similars': 'graus universitaris',
    'graus universitaris de mes de 240 credits i similars': 'graus universitaris',
    'masters universitaris i similars': 'masters universitaris',
    'doctorat universitari': 'doctorat universitari'
}
# Assignem la categoria a cada nivell de formació
df_formacio_genere_clean['categoria_educacio'] = df_formacio_genere_clean['nivell_de_formacio_assolit'].map(categoria_map)
df_formacio_genere_clean.head()

,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor,categoria_educacio
0,2022,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27,analfabetisme
1,2022,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,501,educacio primaria incompleta
2,2022,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,1944,educacio primaria
3,2022,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7527,educacio secundaria
4,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1974,educacio secundaria


In [30]:
df_formacio_genere_clean['categoria_educacio'].unique()

array(['analfabetisme', 'educacio primaria incompleta',
       'educacio primaria', 'educacio secundaria',
       'formacio professional', 'graus universitaris',
       'masters universitaris', 'doctorat universitari'], dtype=object)

In [31]:
# Seleccionem les columnes necessàries
df_formacio_grouped = df_formacio_genere_clean[['comarca', 'sexe', 'valor', 'categoria_educacio']]

In [32]:
# Agrupem les dades per comarca, categoria d'educació i sexe
df_grouped = df_formacio_grouped.groupby(['comarca', 'categoria_educacio', 'sexe'])['valor'].sum().reset_index()

In [33]:
# Comprovem les dades agrupades
df_grouped.head(10)

,comarca,categoria_educacio,sexe,valor
0,alt camp,analfabetisme,dones,60
1,alt camp,analfabetisme,homes,27
2,alt camp,analfabetisme,total,84
3,alt camp,doctorat universitari,dones,126
4,alt camp,doctorat universitari,homes,108
5,alt camp,doctorat universitari,total,234
6,alt camp,educacio primaria,dones,2214
7,alt camp,educacio primaria,homes,1944
8,alt camp,educacio primaria,total,4155
9,alt camp,educacio primaria incompleta,dones,720


In [34]:
# Creem una taula pivote per a cada combinació de categoria i sexe
df_formacio_genere = df_grouped.pivot_table(
    index='comarca', 
    columns=['categoria_educacio', 'sexe'], 
    values='valor', 
    aggfunc='sum'
).reset_index()
df_formacio_genere.columns = df_formacio_genere.columns.to_series().str.join('_')
df_formacio_genere.rename(columns={'comarca_': 'comarca'}, inplace=True)
df_formacio_genere.head()

,comarca,analfabetisme_dones,analfabetisme_homes,analfabetisme_total,doctorat universitari_dones,doctorat universitari_homes,doctorat universitari_total,educacio primaria_dones,educacio primaria_homes,educacio primaria_total,...,educacio secundaria_total,formacio professional_dones,formacio professional_homes,formacio professional_total,graus universitaris_dones,graus universitaris_homes,graus universitaris_total,masters universitaris_dones,masters universitaris_homes,masters universitaris_total
0,alt camp,60,27,84,126,108,234,2214,1944,4155,...,22095,1899,2325,4224,3270,2430,5703,546,366,909
1,alt emporda,183,120,306,351,456,810,8817,8082,16902,...,70659,3738,4005,7740,10440,7527,17967,1332,942,2274
2,alt penedes,126,33,159,363,390,753,6192,4815,11010,...,50634,4662,5217,9879,9318,6876,16191,1365,855,2220
3,alt urgell,18,12,30,90,87,177,1071,981,2055,...,9756,699,876,1572,1917,1260,3174,324,195,522
4,alta ribagorca,6,3,9,18,15,36,210,156,366,...,1761,156,207,363,444,315,756,57,33,87


In [35]:
df_formacio_genere.comarca.unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental'],
      dtype=object)

In [36]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_genere.insert(0, 'codi', df_formacio_genere['comarca'].map(comarques_dict))
df_formacio_genere = df_formacio_genere.sort_values(by='codi').reset_index(drop=True)

In [37]:
df_formacio_genere

,codi,comarca,analfabetisme_dones,analfabetisme_homes,analfabetisme_total,doctorat universitari_dones,doctorat universitari_homes,doctorat universitari_total,educacio primaria_dones,educacio primaria_homes,...,educacio secundaria_total,formacio professional_dones,formacio professional_homes,formacio professional_total,graus universitaris_dones,graus universitaris_homes,graus universitaris_total,masters universitaris_dones,masters universitaris_homes,masters universitaris_total
0,01,alt camp,60,27,84,126,108,234,2214,1944,...,22095,1899,2325,4224,3270,2430,5703,546,366,909
1,02,alt emporda,183,120,306,351,456,810,8817,8082,...,70659,3738,4005,7740,10440,7527,17967,1332,942,2274
2,03,alt penedes,126,33,159,363,390,753,6192,4815,...,50634,4662,5217,9879,9318,6876,16191,1365,855,2220
3,04,alt urgell,18,12,30,90,87,177,1071,981,...,9756,699,876,1572,1917,1260,3174,324,195,522
4,05,alta ribagorca,6,3,9,18,15,36,210,156,...,1761,156,207,363,444,315,756,57,33,87
5,06,anoia,225,54,282,312,321,633,6177,4965,...,61824,5268,5805,11073,8955,6537,15492,1311,816,2127
6,07,bages,234,66,297,522,564,1089,10041,7554,...,85467,7128,7701,14826,14940,11079,26019,2208,1371,3576
7,08,baix camp,792,288,1080,798,768,1563,9588,7641,...,92082,7593,8460,16053,15723,12132,27852,2949,1857,4803
8,09,baix ebre,84,33,117,168,198,369,5334,4287,...,38880,2604,3036,5640,5355,3927,9276,1104,672,1776
9,10,baix emporda,348,132,483,426,528,954,7593,7158,...,65154,4467,4467,8934,12102,9684,21786,1584,993,2577


In [38]:
df_formacio_genere.to_csv('clean_data/2022/df_formacio_genere.csv', index=False)

### Formacio Gènere per Categoría

In [235]:
df_formacio_sexe = pd.read_csv('csv/2022/nivell_formacio_sexe_2022.csv', delimiter=';') 

In [237]:
df_formacio_sexe

,any,comarca o Aran,sexe,nivell de formació assolit,concepte,estat,valor
0,2022,Alt Camp,homes,analfabetisme,població de 15 anys o més,NaN,27
1,2022,Alt Camp,homes,educació primària incompleta,població de 15 anys o més,NaN,501
2,2022,Alt Camp,homes,educació primària,població de 15 anys o més,NaN,1944
3,2022,Alt Camp,homes,primera etapa d’educació secundària i similar,població de 15 anys o més,NaN,7527
4,2022,Alt Camp,homes,"segona etapa d’educació secundària, amb orient...",població de 15 anys o més,NaN,1974
...,...,...,...,...,...,...,...
1543,2022,Catalunya,total,graus universitaris de fins a 240 crèdits i si...,població de 15 anys o més,NaN,725943
1544,2022,Catalunya,total,graus universitaris de més de 240 crèdits i si...,població de 15 anys o més,NaN,656843
1545,2022,Catalunya,total,màsters universitaris i similars,població de 15 anys o més,NaN,205347
1546,2022,Catalunya,total,doctorat universitari,població de 15 anys o més,NaN,86122


In [239]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_genere_clean = clean_dataframe(df_formacio_sexe)
df_formacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1548 non-null   int64  
 1   comarca_o_aran              1548 non-null   object 
 2   sexe                        1548 non-null   object 
 3   nivell_de_formacio_assolit  1548 non-null   object 
 4   concepte                    1548 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1548 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 84.8+ KB


In [241]:
# Corregir nom columna comarca i filtrar per 'total' i només per l'any 2022
df_formacio_genere_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['nivell_de_formacio_assolit'] != 'total']
df_formacio_genere_clean = df_formacio_genere_clean[df_formacio_genere_clean['any'] == 2022]

In [243]:
# Filtrar per comarques
df_formacio_genere_clean = filtrar_comarques(df_formacio_genere_clean, 'comarca')

In [245]:
# Comprovem els canvis realitzats
df_formacio_genere_clean.info()
display(df_formacio_genere_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   any                         1386 non-null   int64  
 1   comarca                     1386 non-null   object 
 2   sexe                        1386 non-null   object 
 3   nivell_de_formacio_assolit  1386 non-null   object 
 4   concepte                    1386 non-null   object 
 5   estat                       0 non-null      float64
 6   valor                       1386 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 75.9+ KB


,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor
0,2022,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27
1,2022,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,501
2,2022,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,1944
3,2022,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7527
4,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1974
5,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,2454
6,2022,alt camp,homes,ensenyaments de grau superior de formacio prof...,poblacio de 15 anys o mes,NaN,2325
7,2022,alt camp,homes,graus universitaris de fins a 240 credits i si...,poblacio de 15 anys o mes,NaN,1350
8,2022,alt camp,homes,graus universitaris de mes de 240 credits i si...,poblacio de 15 anys o mes,NaN,1080
9,2022,alt camp,homes,masters universitaris i similars,poblacio de 15 anys o mes,NaN,366


In [247]:
df_formacio_genere_clean['nivell_de_formacio_assolit'].unique()

array(['analfabetisme', 'educacio primaria incompleta',
       'educacio primaria',
       "primera etapa d'educacio secundaria i similar",
       "segona etapa d'educacio secundaria, amb orientacio general",
       "segona etapa d'educacio secundaria, amb orientacio professional",
       'ensenyaments de grau superior de formacio professional i similars',
       'graus universitaris de fins a 240 credits i similars',
       'graus universitaris de mes de 240 credits i similars',
       'masters universitaris i similars', 'doctorat universitari'],
      dtype=object)

Agrupem els següents grups per homes, dones i total:  
* sense educació
    * analfabetisme
    * educació primària incompleta  
* educació obligatoria
    *  educació primaria
    *  primera etapa d’educació secundària i similarmària
    *  segona etapa d’educació secundària, amb orientació general
    *  segona etapa d’educació secundària, amb orientació professionales
* estudis superiors
    * ensenyaments de grau superior de formació professional i similars
    * graus universitaris de fins a 240 crèdits i similars
    * graus universitaris de més de 240 crèdits i similars
    * màsters universitaris i similars
    * doctorat universitari

In [250]:
# Creem el mapeig de categories educatives
categoria_map = {
    'analfabetisme': 'sense_educacio',
    'educacio primaria incompleta': 'sense_educacio',
    'educacio primaria': 'educacio_obligatoria',
    'primera etapa d\'educacio secundaria i similar': 'educacio_obligatoria',
    'segona etapa d\'educacio secundaria, amb orientacio general': 'educacio_obligatoria',
    'segona etapa d\'educacio secundaria, amb orientacio professional': 'educacio_obligatoria',
    'ensenyaments de grau superior de formacio professional i similars': 'estudis_superiors',
    'graus universitaris de fins a 240 credits i similars': 'estudis_superiors',
    'graus universitaris de mes de 240 credits i similars': 'estudis_superiors',
    'masters universitaris i similars': 'estudis_superiors',
    'doctorat universitari': 'estudis_superiors'
}
# Assignem la categoria a cada nivell de formació
df_formacio_genere_clean['tipus_estudis'] = df_formacio_genere_clean['nivell_de_formacio_assolit'].map(categoria_map)
df_formacio_genere_clean.head()

,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor,tipus_estudis
0,2022,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27,sense_educacio
1,2022,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,501,sense_educacio
2,2022,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,1944,educacio_obligatoria
3,2022,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7527,educacio_obligatoria
4,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1974,educacio_obligatoria


In [252]:
df_formacio_genere_clean

,any,comarca,sexe,nivell_de_formacio_assolit,concepte,estat,valor,tipus_estudis
0,2022,alt camp,homes,analfabetisme,poblacio de 15 anys o mes,NaN,27,sense_educacio
1,2022,alt camp,homes,educacio primaria incompleta,poblacio de 15 anys o mes,NaN,501,sense_educacio
2,2022,alt camp,homes,educacio primaria,poblacio de 15 anys o mes,NaN,1944,educacio_obligatoria
3,2022,alt camp,homes,primera etapa d'educacio secundaria i similar,poblacio de 15 anys o mes,NaN,7527,educacio_obligatoria
4,2022,alt camp,homes,"segona etapa d'educacio secundaria, amb orient...",poblacio de 15 anys o mes,NaN,1974,educacio_obligatoria
...,...,...,...,...,...,...,...,...
1381,2022,valles oriental,total,ensenyaments de grau superior de formacio prof...,poblacio de 15 anys o mes,NaN,39432,estudis_superiors
1382,2022,valles oriental,total,graus universitaris de fins a 240 credits i si...,poblacio de 15 anys o mes,NaN,35220,estudis_superiors
1383,2022,valles oriental,total,graus universitaris de mes de 240 credits i si...,poblacio de 15 anys o mes,NaN,27381,estudis_superiors
1384,2022,valles oriental,total,masters universitaris i similars,poblacio de 15 anys o mes,NaN,8643,estudis_superiors


In [254]:
df_formacio_genere_clean['tipus_estudis'].unique()

array(['sense_educacio', 'educacio_obligatoria', 'estudis_superiors'],
      dtype=object)

In [256]:
# Seleccionem les columnes necessàries
df_formacio_grouped = df_formacio_genere_clean[['comarca', 'sexe', 'valor', 'tipus_estudis']]

In [258]:
# Agrupem les dades per comarca, categoria d'educació i sexe
df_formacio_grouped = df_formacio_grouped.groupby(['comarca', 'tipus_estudis', 'sexe'])['valor'].sum().reset_index()
df_formacio_grouped.head(10)

,comarca,tipus_estudis,sexe,valor
0,alt camp,educacio_obligatoria,dones,12360
1,alt camp,educacio_obligatoria,homes,13899
2,alt camp,educacio_obligatoria,total,26250
3,alt camp,estudis_superiors,dones,5841
4,alt camp,estudis_superiors,homes,5229
5,alt camp,estudis_superiors,total,11070
6,alt camp,sense_educacio,dones,780
7,alt camp,sense_educacio,homes,528
8,alt camp,sense_educacio,total,1305
9,alt emporda,educacio_obligatoria,dones,42156


In [260]:
# Eliminar files on 'sexe' es igual a 'total'
df_grouped_genere = df_formacio_grouped[df_formacio_grouped['sexe'] != 'total']
df_grouped_genere.head()

,comarca,tipus_estudis,sexe,valor
0,alt camp,educacio_obligatoria,dones,12360
1,alt camp,educacio_obligatoria,homes,13899
3,alt camp,estudis_superiors,dones,5841
4,alt camp,estudis_superiors,homes,5229
6,alt camp,sense_educacio,dones,780


In [262]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_grouped_genere.insert(0, 'codi', df_grouped_genere['comarca'].map(comarques_dict))
df_grouped_genere = df_grouped_genere.sort_values(by='codi').reset_index(drop=True)
df_grouped_genere.head()

,codi,comarca,tipus_estudis,sexe,valor
0,01,alt camp,educacio_obligatoria,dones,12360
1,01,alt camp,educacio_obligatoria,homes,13899
2,01,alt camp,estudis_superiors,dones,5841
3,01,alt camp,estudis_superiors,homes,5229
4,01,alt camp,sense_educacio,dones,780


In [264]:
df_grouped_genere.head(30)

,codi,comarca,tipus_estudis,sexe,valor
0,01,alt camp,educacio_obligatoria,dones,12360
1,01,alt camp,educacio_obligatoria,homes,13899
2,01,alt camp,estudis_superiors,dones,5841
3,01,alt camp,estudis_superiors,homes,5229
4,01,alt camp,sense_educacio,dones,780
5,01,alt camp,sense_educacio,homes,528
6,02,alt emporda,educacio_obligatoria,dones,42156
7,02,alt emporda,educacio_obligatoria,homes,45408
8,02,alt emporda,estudis_superiors,dones,15861
9,02,alt emporda,estudis_superiors,homes,12930


In [266]:
df_grouped_genere.to_csv('clean_data/2022/formacio_genere_pivot.csv', index=False)

#### Formació d'Adults<a class="anchor" id="formacio_adults"></a>

In [40]:
df_formacio_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    43 non-null     object
 1   Centres                    43 non-null     int64 
 2   Personal docent            43 non-null     int64 
 3   Alumnes (homes). 16-24     43 non-null     int64 
 4   Alumnes (homes). 25-64     43 non-null     int64 
 5   Alumnes (homes). 65 i més  43 non-null     int64 
 6   Alumnes (homes). Total     43 non-null     int64 
 7   Alumnes (dones). 16-24     43 non-null     int64 
 8   Alumnes (dones). 25-64     43 non-null     int64 
 9   Alumnes (dones). 65 i més  43 non-null     int64 
 10  Alumnes (dones). Total     43 non-null     int64 
 11  Alumnes (total). 16-24     43 non-null     int64 
 12  Alumnes (total). 25-64     43 non-null     int64 
 13  Alumnes (total). 65 i més  43 non-null     int64 
 14  Alumnes (tot

In [41]:
df_formacio_adults.head(10)

,comarca,Centres,Personal docent,Alumnes (homes). 16-24,Alumnes (homes). 25-64,Alumnes (homes). 65 i més,Alumnes (homes). Total,Alumnes (dones). 16-24,Alumnes (dones). 25-64,Alumnes (dones). 65 i més,Alumnes (dones). Total,Alumnes (total). 16-24,Alumnes (total). 25-64,Alumnes (total). 65 i més,Alumnes (total). Total
0,Alt Camp,1,7,24,59,8,91,24,155,34,213,48,214,42,304
1,Alt Empordà,4,38,235,616,12,863,66,317,17,400,301,933,29,1263
2,Alt Penedès,2,13,82,124,18,224,52,217,22,291,134,341,40,515
3,Alt Urgell,1,6,19,103,18,140,23,221,37,281,42,324,55,421
4,Alta Ribagorça,1,1,0,0,0,0,0,20,0,20,0,20,0,20
5,Anoia,1,5,27,58,2,87,20,152,0,172,47,210,2,259
6,Aran,1,3,4,44,4,52,10,124,8,142,14,168,12,194
7,Bages,3,38,110,583,36,729,76,444,37,557,186,1027,73,1286
8,Baix Camp,4,38,142,266,39,447,118,684,78,880,260,950,117,1327
9,Baix Ebre,2,13,42,63,25,130,58,219,40,317,100,282,65,447


In [42]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_adults_clean = clean_dataframe(df_formacio_adults)
df_formacio_adults_clean.head()

,comarca,centres,personal_docent,alumnes_(homes)._16-24,alumnes_(homes)._25-64,alumnes_(homes)._65_i_mes,alumnes_(homes)._total,alumnes_(dones)._16-24,alumnes_(dones)._25-64,alumnes_(dones)._65_i_mes,alumnes_(dones)._total,alumnes_(total)._16-24,alumnes_(total)._25-64,alumnes_(total)._65_i_mes,alumnes_(total)._total
0,alt camp,1,7,24,59,8,91,24,155,34,213,48,214,42,304
1,alt emporda,4,38,235,616,12,863,66,317,17,400,301,933,29,1263
2,alt penedes,2,13,82,124,18,224,52,217,22,291,134,341,40,515
3,alt urgell,1,6,19,103,18,140,23,221,37,281,42,324,55,421
4,alta ribagorca,1,1,0,0,0,0,0,20,0,20,0,20,0,20


In [43]:
# Filtrar per comarques
df_formacio_adults_clean = filtrar_comarques(df_formacio_adults_clean, 'comarca')
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    42 non-null     object
 1   centres                    42 non-null     int64 
 2   personal_docent            42 non-null     int64 
 3   alumnes_(homes)._16-24     42 non-null     int64 
 4   alumnes_(homes)._25-64     42 non-null     int64 
 5   alumnes_(homes)._65_i_mes  42 non-null     int64 
 6   alumnes_(homes)._total     42 non-null     int64 
 7   alumnes_(dones)._16-24     42 non-null     int64 
 8   alumnes_(dones)._25-64     42 non-null     int64 
 9   alumnes_(dones)._65_i_mes  42 non-null     int64 
 10  alumnes_(dones)._total     42 non-null     int64 
 11  alumnes_(total)._16-24     42 non-null     int64 
 12  alumnes_(total)._25-64     42 non-null     int64 
 13  alumnes_(total)._65_i_mes  42 non-null     int64 
 14  alumnes_(tot

In [44]:
# Suma d'alumnes majors de 25 per a homes, dones i el total
df_formacio_adults_clean.loc[:, 'alumnes_homes_+25_anys'] = df_formacio_adults_clean['alumnes_(homes)._25-64'] + df_formacio_adults['alumnes_(homes)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_dones_+25_anys'] = df_formacio_adults_clean['alumnes_(dones)._25-64'] + df_formacio_adults['alumnes_(dones)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_total_+25_anys'] = df_formacio_adults_clean['alumnes_(total)._25-64'] + df_formacio_adults['alumnes_(total)._65_i_mes']


# Convertir les columnes +25 anys a tipus int
df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']] = (
    df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']].astype(int)
)

# Seleccionar i renombrar columnes
df_formacio_adults_clean = df_formacio_adults_clean[['comarca',
                                         'alumnes_(homes)._16-24',
                                         'alumnes_(dones)._16-24',
                                         'alumnes_(total)._16-24',
                                         'alumnes_homes_+25_anys',
                                         'alumnes_dones_+25_anys',
                                         'alumnes_total_+25_anys']].rename(columns={
    'alumnes_(homes)._16-24': 'alumnes_homes_16_24_anys',
    'alumnes_(dones)._16-24': 'alumnes_dones_16_24_anys',
    'alumnes_(total)._16-24': 'alumnes_total_16_24_anys'
})
# Dataset resultant
df_formacio_adults_clean.head()


,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,alt camp,24,24,48,67,189,256
1,alt emporda,235,66,301,628,334,962
2,alt penedes,82,52,134,142,239,381
3,alt urgell,19,23,42,121,258,379
4,alta ribagorca,0,0,0,0,20,20


In [45]:
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   comarca                   42 non-null     object
 1   alumnes_homes_16_24_anys  42 non-null     int64 
 2   alumnes_dones_16_24_anys  42 non-null     int64 
 3   alumnes_total_16_24_anys  42 non-null     int64 
 4   alumnes_homes_+25_anys    42 non-null     int32 
 5   alumnes_dones_+25_anys    42 non-null     int32 
 6   alumnes_total_+25_anys    42 non-null     int32 
dtypes: int32(3), int64(3), object(1)
memory usage: 1.9+ KB


In [46]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_adults_clean.insert(0, 'codi', df_formacio_adults_clean['comarca'].map(comarques_dict))
df_formacio_adults_clean = df_formacio_adults_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_adults_clean.head()

,codi,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,01,alt camp,24,24,48,67,189,256
1,02,alt emporda,235,66,301,628,334,962
2,03,alt penedes,82,52,134,142,239,381
3,04,alt urgell,19,23,42,121,258,379
4,05,alta ribagorca,0,0,0,0,20,20


In [47]:
df_formacio_adults_clean.to_csv('clean_data/2022/df_formacio_adults.csv', index=False)

### Dades Població<a class="anchor" id="poblacio"></a>

#### Població per Gènere<a class="anchor" id="poblacio_genere"></a>

In [50]:
df_poblacio_genere = pd.read_csv('csv/2021/poblacio.csv', delimiter=';')

In [51]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_poblacio_genere_clean = clean_dataframe(df_poblacio_genere)
df_poblacio_genere_clean.head()

,any,comarca_o_aran,sexe,concepte,estat,valor
0,1975,alt camp,homes,poblacio,NaN,15242.0
1,1975,alt camp,dones,poblacio,NaN,15869.0
2,1975,alt camp,total,poblacio,NaN,31111.0
3,1975,alt emporda,homes,poblacio,NaN,38608.0
4,1975,alt emporda,dones,poblacio,NaN,38993.0


In [52]:
df_poblacio_genere_clean['comarca_o_aran'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya'], dtype=object)

In [53]:
# Canviar el nom de la columna 'comarca_o_aran' a 'comarca' per simplificar el nom
df_poblacio_genere_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
# Filtrar el DataFrame per excloure les files on la comarca és 'catalunya', ja que només volem les dades a nivell de comarques individuals
df_poblacio_genere_clean = df_poblacio_genere_clean[df_poblacio_genere_clean['comarca'] != 'catalunya']
# Filtrar les dades per mantenir únicament les corresponents a l'any 2022
df_poblacio_genere_clean = df_poblacio_genere_clean[df_poblacio_genere_clean['any'] == 2022]

In [54]:
# Filtrem per comarques
df_poblacio_genere_clean = filtrar_comarques(df_poblacio_genere_clean, 'comarca')

In [55]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   any       126 non-null    int64  
 1   comarca   126 non-null    object 
 2   sexe      126 non-null    object 
 3   concepte  126 non-null    object 
 4   estat     0 non-null      object 
 5   valor     126 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 6.0+ KB


In [56]:
df_poblacio_genere_clean.head()

,any,comarca,sexe,concepte,estat,valor
0,2022,alt camp,homes,poblacio,NaN,23132.0
1,2022,alt camp,dones,poblacio,NaN,22337.0
2,2022,alt camp,total,poblacio,NaN,45469.0
3,2022,alt emporda,homes,poblacio,NaN,72257.0
4,2022,alt emporda,dones,poblacio,NaN,71693.0


In [57]:
# Crear una taula pivot amb 'comarca' com a índex, 'sexe' com a columnes i suma de 'valor'
df_poblacio_genere_clean = df_poblacio_genere_clean.pivot_table(
    index='comarca', 
    columns=['sexe'], 
    values='valor', 
    aggfunc='sum'
).reset_index()
# Renombrar les columnes a 'comarca', 'dones', 'homes', i 'total'
df_poblacio_genere_clean.columns = ['comarca', 'dones', 'homes', 'total']
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,22337.0,23132.0,45469.0
1,alt emporda,71693.0,72257.0,143950.0
2,alt penedes,55374.0,55436.0,110810.0
3,alt urgell,10217.0,10241.0,20458.0
4,alta ribagorca,1927.0,2062.0,3989.0


In [58]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_genere_clean.insert(0, 'codi', df_poblacio_genere_clean['comarca'].map(comarques_dict))
df_poblacio_genere_clean = df_poblacio_genere_clean.sort_values(by='codi').reset_index(drop=True)
# Convertir les columnes 'dones', 'homes' i 'total' a tipus int
df_poblacio_genere_clean[['dones', 'homes', 'total']] = df_poblacio_genere_clean[['dones', 'homes', 'total']].astype(int)
df_poblacio_genere_clean.head()

,codi,comarca,dones,homes,total
0,01,alt camp,22337,23132,45469
1,02,alt emporda,71693,72257,143950
2,03,alt penedes,55374,55436,110810
3,04,alt urgell,10217,10241,20458
4,05,alta ribagorca,1927,2062,3989


In [59]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   dones    42 non-null     int32 
 3   homes    42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [60]:
df_poblacio_genere_clean.to_csv('clean_data/2022/df_poblacio_genere.csv', index=False)

#### Població per Edat<a class="anchor" id="poblacio_edat"></a>

In [62]:
df_poblacio_edat = pd.read_csv('csv/2022/poblacio_edat.csv', delimiter=';')

In [63]:
df_poblacio_edat.head() 

,any,comarca o Aran,edat,sexe,concepte,estat,valor
0,2011,Alt Camp,0 anys,homes,població,NaN,266.0
1,2011,Alt Camp,0 anys,dones,població,NaN,272.0
2,2011,Alt Camp,0 anys,total,població,NaN,538.0
3,2011,Alt Camp,1 any,homes,població,NaN,277.0
4,2011,Alt Camp,1 any,dones,població,NaN,252.0


In [64]:
# Netejar el DataFrame 'df_poblacio_edat' utilitzant la funció 'clean_dataframe'
df_poblacio_edat_clean = clean_dataframe(df_poblacio_edat)
df_poblacio_edat_clean.head()

,any,comarca_o_aran,edat,sexe,concepte,estat,valor
0,2011,alt camp,0 anys,homes,poblacio,NaN,266.0
1,2011,alt camp,0 anys,dones,poblacio,NaN,272.0
2,2011,alt camp,0 anys,total,poblacio,NaN,538.0
3,2011,alt camp,1 any,homes,poblacio,NaN,277.0
4,2011,alt camp,1 any,dones,poblacio,NaN,252.0


In [65]:
# Filtrar les comarques utilitzant la funció 'filtrar_comarques'
df_poblacio_edat_clean = filtrar_comarques(df_poblacio_edat_clean, 'comarca_o_aran')

In [66]:
# Renombrar la columna 'comarca_o_aran' a 'comarca'
df_poblacio_edat_clean.rename(columns={'comarca_o_aran': 'comarca'}, inplace=True)
# Filtrar les dades per mantenir només les corresponents a l'any 2022
df_poblacio_edat_clean = df_poblacio_edat_clean[df_poblacio_edat_clean['any'] == 2022]

In [67]:
df_poblacio_edat_clean.head()

,any,comarca,edat,sexe,concepte,estat,valor
25704,2022,alt camp,0 anys,homes,poblacio,NaN,176.0
25705,2022,alt camp,0 anys,dones,poblacio,NaN,165.0
25706,2022,alt camp,0 anys,total,poblacio,NaN,341.0
25707,2022,alt camp,1 any,homes,poblacio,NaN,182.0
25708,2022,alt camp,1 any,dones,poblacio,NaN,184.0


In [68]:
# Filtrar les dades per incloure només edats entre 16 i 24 anys
df_16_24 = df_poblacio_edat_clean[df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])]

# Agrupar les dades per 'comarca' i 'edat', sumant els valors
df_age_grouped = df_16_24.groupby(['comarca', 'edat'])['valor'].sum().reset_index()

# Crear una taula pivot amb 'comarca' com a índex i les edats com a columnes
df_pivoted = df_age_grouped.pivot_table(
    index='comarca', 
    columns='edat', 
    values='valor', 
    aggfunc='sum', 
    fill_value=0
)

# Afegir una columna amb la suma total de la població entre 16 i 24 anys per comarca
df_pivoted['poblacio_16_a_24_anys'] = df_pivoted.sum(axis=1)
df_pivoted.head()

edat,16 anys,17 anys,18 anys,19 anys,20 anys,21 anys,22 anys,23 anys,24 anys,poblacio_16_a_24_anys
comarca,,,,,,,,,,
alt camp,984.0,962.0,1066.0,1006.0,914.0,1028.0,902.0,938.0,872.0,8672.0
alt emporda,3218.0,3038.0,3042.0,2956.0,3048.0,3084.0,3024.0,2892.0,3058.0,27360.0
alt penedes,2714.0,2740.0,2560.0,2464.0,2472.0,2360.0,2188.0,2194.0,2154.0,21846.0
alt urgell,410.0,406.0,346.0,372.0,382.0,400.0,436.0,426.0,394.0,3572.0
alta ribagorca,116.0,68.0,46.0,64.0,76.0,74.0,76.0,76.0,80.0,676.0


In [69]:
# Filtrar el DataFrame per la franja d'edat 16-24 anys i 'sexe' = 'total'
df_16_24_total = df_poblacio_edat_clean[(df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])) & (df_poblacio_edat_clean['sexe'] == 'total')]

# Agrupar per 'comarca' i sumar els valors per obtenir la població total
df_16_24_total_grouped = df_16_24_total.groupby('comarca')['valor'].sum().reset_index()
df_16_24_total_grouped.rename(columns={'valor': 'poblacio_total_16_a_24_anys'}, inplace=True)

# Filtrar el DataFrame per homes ('homes') i dones ('dones') dins la franja d'edat 16-24 anys
df_16_24_homes = df_poblacio_edat_clean[(df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])) & (df_poblacio_edat_clean['sexe'] == 'homes')]
df_16_24_dones = df_poblacio_edat_clean[(df_poblacio_edat_clean['edat'].isin(['16 anys', '17 anys', '18 anys', '19 anys', '20 anys', '21 anys', '22 anys', '23 anys', '24 anys'])) & (df_poblacio_edat_clean['sexe'] == 'dones')]

# Agrupar per 'comarca' i sumar els valors per a homes
df_16_24_homes_grouped = df_16_24_homes.groupby('comarca')['valor'].sum().reset_index()
df_16_24_homes_grouped.rename(columns={'valor': 'poblacio_homes_16_a_24_anys'}, inplace=True)

# Agrupar per 'comarca' i sumar els valors per a dones
df_16_24_dones_grouped = df_16_24_dones.groupby('comarca')['valor'].sum().reset_index()
df_16_24_dones_grouped.rename(columns={'valor': 'poblacio_dones_16_a_24_anys'}, inplace=True)

# Fusionar els DataFrames per obtenir la població total, homes i dones per comarca
df_poblacio_edat_final = df_16_24_total_grouped.merge(df_16_24_homes_grouped, on='comarca', how='left')
df_poblacio_edat_final = df_poblacio_edat_final.merge(df_16_24_dones_grouped, on='comarca', how='left')

In [70]:
df_poblacio_edat_final.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,4336.0,2330.0,2006.0
1,alt emporda,13680.0,7019.0,6661.0
2,alt penedes,10923.0,5668.0,5255.0
3,alt urgell,1786.0,949.0,837.0
4,alta ribagorca,338.0,172.0,166.0


In [71]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_edat_final.insert(0, 'codi', df_poblacio_edat_final['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_edat_final = df_poblacio_edat_final.sort_values(by='codi').reset_index(drop=True)
# Convertir les columnes numériques a tipus int
df_poblacio_edat_final[['poblacio_total_16_a_24_anys', 'poblacio_homes_16_a_24_anys', 'poblacio_dones_16_a_24_anys']] = df_poblacio_edat_final[['poblacio_total_16_a_24_anys', 'poblacio_homes_16_a_24_anys', 'poblacio_dones_16_a_24_anys']].astype(int)
df_poblacio_edat_final.head()

,codi,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,01,alt camp,4336,2330,2006
1,02,alt emporda,13680,7019,6661
2,03,alt penedes,10923,5668,5255
3,04,alt urgell,1786,949,837
4,05,alta ribagorca,338,172,166


In [72]:
df_poblacio_edat_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   codi                         42 non-null     object
 1   comarca                      42 non-null     object
 2   poblacio_total_16_a_24_anys  42 non-null     int32 
 3   poblacio_homes_16_a_24_anys  42 non-null     int32 
 4   poblacio_dones_16_a_24_anys  42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [73]:
df_poblacio_edat_final.to_csv('clean_data/2022/df_poblacio_edat.csv', index=False)

#### Població Activa (16-24 anys)<a class="anchor" id="poblacio_activa"></a>

In [75]:
df_poblacio_activa = pd.read_csv('csv/2022/poblacio_activa_2022.csv', delimiter=';', skiprows=6)

In [76]:
df_poblacio_activa.head()

,Unnamed: 0,De 16 a 19 anys,De 20 a 24 anys,De 25 a 29 anys,De 30 a 34 anys,De 35 a 39 anys,De 40 a 44 anys,De 45 a 49 anys,De 50 a 54 anys,De 55 a 59 anys,De 60 a 64 anys,65 anys o més,Total
0,Alt Camp,270,1269,1746,2055,2373,3177,3222,2955,2661,1719,333,21786
1,Alt Empordà,816,3912,5310,5865,7137,9201,9435,8499,7599,5211,1305,64287
2,Alt Penedès,741,3174,4077,4410,5739,8112,8994,7611,6282,3894,891,53922
3,Alt Urgell,105,504,714,780,879,1137,1230,1119,1074,813,174,8532
4,Alta Ribagorça,24,87,153,183,210,273,318,291,231,180,45,2001


In [77]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_poblacio_activa = df_poblacio_activa.rename(columns={'Unnamed: 0': 'comarca'})
df_poblacio_activa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   comarca          43 non-null     object
 1   De 16 a 19 anys  43 non-null     int64 
 2   De 20 a 24 anys  43 non-null     int64 
 3   De 25 a 29 anys  43 non-null     int64 
 4   De 30 a 34 anys  43 non-null     int64 
 5   De 35 a 39 anys  43 non-null     int64 
 6   De 40 a 44 anys  43 non-null     int64 
 7   De 45 a 49 anys  43 non-null     int64 
 8   De 50 a 54 anys  43 non-null     int64 
 9   De 55 a 59 anys  43 non-null     int64 
 10  De 60 a 64 anys  43 non-null     int64 
 11  65 anys o més    43 non-null     int64 
 12  Total            43 non-null     int64 
dtypes: int64(12), object(1)
memory usage: 4.5+ KB


In [78]:
# Netejar el DataFrame 'df_poblacio_activa' utilitzant la funció 'clean_dataframe'
df_poblacio_activa_clean = clean_dataframe(df_poblacio_activa)
df_poblacio_activa_clean.head()

,comarca,de_16_a_19_anys,de_20_a_24_anys,de_25_a_29_anys,de_30_a_34_anys,de_35_a_39_anys,de_40_a_44_anys,de_45_a_49_anys,de_50_a_54_anys,de_55_a_59_anys,de_60_a_64_anys,65_anys_o_mes,total
0,alt camp,270,1269,1746,2055,2373,3177,3222,2955,2661,1719,333,21786
1,alt emporda,816,3912,5310,5865,7137,9201,9435,8499,7599,5211,1305,64287
2,alt penedes,741,3174,4077,4410,5739,8112,8994,7611,6282,3894,891,53922
3,alt urgell,105,504,714,780,879,1137,1230,1119,1074,813,174,8532
4,alta ribagorca,24,87,153,183,210,273,318,291,231,180,45,2001


In [79]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_poblacio_activa_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya'], dtype=object)

In [80]:
# Filtrar les dades del DataFrame per obtenir només les comarques necesaries
df_poblacio_activa_clean = filtrar_comarques(df_poblacio_activa_clean, 'comarca')
df_poblacio_activa_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   comarca          42 non-null     object
 1   de_16_a_19_anys  42 non-null     int64 
 2   de_20_a_24_anys  42 non-null     int64 
 3   de_25_a_29_anys  42 non-null     int64 
 4   de_30_a_34_anys  42 non-null     int64 
 5   de_35_a_39_anys  42 non-null     int64 
 6   de_40_a_44_anys  42 non-null     int64 
 7   de_45_a_49_anys  42 non-null     int64 
 8   de_50_a_54_anys  42 non-null     int64 
 9   de_55_a_59_anys  42 non-null     int64 
 10  de_60_a_64_anys  42 non-null     int64 
 11  65_anys_o_mes    42 non-null     int64 
 12  total            42 non-null     int64 
dtypes: int64(12), object(1)
memory usage: 4.4+ KB


In [81]:
# Crear una nova columna 'poblacio_activa_16_a_24_anys' sumant les columnes 'de_16_a_19_anys' i 'de_20_a_24_anys'
df_poblacio_activa_clean.loc[:, 'poblacio_activa_16_a_24_anys'] = (
    df_poblacio_activa_clean['de_16_a_19_anys'] + df_poblacio_activa_clean['de_20_a_24_anys']
)
df_poblacio_activa_clean.head()

,comarca,de_16_a_19_anys,de_20_a_24_anys,de_25_a_29_anys,de_30_a_34_anys,de_35_a_39_anys,de_40_a_44_anys,de_45_a_49_anys,de_50_a_54_anys,de_55_a_59_anys,de_60_a_64_anys,65_anys_o_mes,total,poblacio_activa_16_a_24_anys
0,alt camp,270,1269,1746,2055,2373,3177,3222,2955,2661,1719,333,21786,1539
1,alt emporda,816,3912,5310,5865,7137,9201,9435,8499,7599,5211,1305,64287,4728
2,alt penedes,741,3174,4077,4410,5739,8112,8994,7611,6282,3894,891,53922,3915
3,alt urgell,105,504,714,780,879,1137,1230,1119,1074,813,174,8532,609
4,alta ribagorca,24,87,153,183,210,273,318,291,231,180,45,2001,111


In [82]:
# Seleccionar només les columnes 'comarca' i 'poblacio_activa_16_a_24_anys'
df_poblacio_activa = df_poblacio_activa_clean[['comarca','poblacio_activa_16_a_24_anys']]
df_poblacio_activa.head()

,comarca,poblacio_activa_16_a_24_anys
0,alt camp,1539
1,alt emporda,4728
2,alt penedes,3915
3,alt urgell,609
4,alta ribagorca,111


In [83]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_activa.insert(0, 'codi', df_poblacio_activa['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_activa = df_poblacio_activa.sort_values(by='codi').reset_index(drop=True)
df_poblacio_activa.head()

,codi,comarca,poblacio_activa_16_a_24_anys
0,01,alt camp,1539
1,02,alt emporda,4728
2,03,alt penedes,3915
3,04,alt urgell,609
4,05,alta ribagorca,111


In [84]:
# Fusionar els DataFrames 'df_poblacio_activa' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_activa_merged = df_poblacio_activa.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_activa_merged.head()

,codi,comarca,poblacio_activa_16_a_24_anys,total
0,01,alt camp,1539,45469
1,02,alt emporda,4728,143950
2,03,alt penedes,3915,110810
3,04,alt urgell,609,20458
4,05,alta ribagorca,111,3989


In [85]:
# Percentatge de població activa (16-24 anys) respecte al total
df_poblacio_activa_merged['pct_activa'] = (df_poblacio_activa_merged['poblacio_activa_16_a_24_anys'] / df_poblacio_activa_merged['total'] * 100).round(2)

# Ràtio de població activa (16-24 anys) respecte al total
df_poblacio_activa_merged['ratio_activa'] = (df_poblacio_activa_merged['poblacio_activa_16_a_24_anys'] / df_poblacio_activa_merged['total'])
df_poblacio_activa_merged.head()

,codi,comarca,poblacio_activa_16_a_24_anys,total,pct_activa,ratio_activa
0,01,alt camp,1539,45469,3.38,0.033847
1,02,alt emporda,4728,143950,3.28,0.032845
2,03,alt penedes,3915,110810,3.53,0.035331
3,04,alt urgell,609,20458,2.98,0.029768
4,05,alta ribagorca,111,3989,2.78,0.027827


In [86]:
df_poblacio_activa_merged.to_csv('clean_data/2022/df_poblacio_activa.csv', index=False)

#### Població Inactiva<a class="anchor" id="poblacio_inactiva"></a>

In [88]:
df_poblacio_inactiva = pd.read_csv('csv/2022/poblacio_inactiva_2022.csv', delimiter=';', skiprows=5)

In [89]:
df_poblacio_inactiva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0                                       55 non-null     object
 1   Població pensionista de jubilació, prejubilació  55 non-null     int64 
 2   Població pensionista d'invalidesa                55 non-null     int64 
 3   Població estudiant                               55 non-null     int64 
 4   Població amb una altra situació d'inactivitat    55 non-null     int64 
 5   Total població inactiva                          55 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.7+ KB


In [90]:
df_poblacio_inactiva.head()

,Unnamed: 0,"Població pensionista de jubilació, prejubilació",Població pensionista d'invalidesa,Població estudiant,Població amb una altra situació d'inactivitat,Total població inactiva
0,Alt Camp,7558,1094,2090,5614,16356
1,Alt Empordà,20492,2163,5896,27505,56056
2,Alt Penedès,16105,2289,4900,14457,37751
3,Alt Urgell,3528,477,893,4222,9120
4,Alta Ribagorça,592,102,195,569,1458


In [91]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_poblacio_inactiva = df_poblacio_inactiva.rename(columns={'Unnamed: 0': 'comarca'})
df_poblacio_inactiva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   comarca                                          55 non-null     object
 1   Població pensionista de jubilació, prejubilació  55 non-null     int64 
 2   Població pensionista d'invalidesa                55 non-null     int64 
 3   Població estudiant                               55 non-null     int64 
 4   Població amb una altra situació d'inactivitat    55 non-null     int64 
 5   Total població inactiva                          55 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.7+ KB


In [92]:
df_poblacio_inactiva.head()

,comarca,"Població pensionista de jubilació, prejubilació",Població pensionista d'invalidesa,Població estudiant,Població amb una altra situació d'inactivitat,Total població inactiva
0,Alt Camp,7558,1094,2090,5614,16356
1,Alt Empordà,20492,2163,5896,27505,56056
2,Alt Penedès,16105,2289,4900,14457,37751
3,Alt Urgell,3528,477,893,4222,9120
4,Alta Ribagorça,592,102,195,569,1458


In [93]:
# Netejar el DataFrame 'df_poblacio_inactiva' amb la funció 'clean_dataframe'
df_poblacio_inactiva_clean = clean_dataframe(df_poblacio_inactiva)
df_poblacio_inactiva_clean.head()

,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva
0,alt camp,7558,1094,2090,5614,16356
1,alt emporda,20492,2163,5896,27505,56056
2,alt penedes,16105,2289,4900,14457,37751
3,alt urgell,3528,477,893,4222,9120
4,alta ribagorca,592,102,195,569,1458


In [94]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_poblacio_inactiva_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [95]:
# Filtrar les comarques en el DataFrame netejat
df_poblacio_inactiva_clean = filtrar_comarques(df_poblacio_inactiva_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_poblacio_inactiva_clean['comarca'].unique())

42

In [96]:
df_poblacio_inactiva_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental'],
      dtype=object)

In [97]:
df_poblacio_inactiva_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   comarca                                          42 non-null     object
 1   poblacio_pensionista_de_jubilacio,_prejubilacio  42 non-null     int64 
 2   poblacio_pensionista_d'invalidesa                42 non-null     int64 
 3   poblacio_estudiant                               42 non-null     int64 
 4   poblacio_amb_una_altra_situacio_d'inactivitat    42 non-null     int64 
 5   total_poblacio_inactiva                          42 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ KB


In [98]:
df_poblacio_inactiva_clean.head()

,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva
0,alt camp,7558,1094,2090,5614,16356
1,alt emporda,20492,2163,5896,27505,56056
2,alt penedes,16105,2289,4900,14457,37751
3,alt urgell,3528,477,893,4222,9120
4,alta ribagorca,592,102,195,569,1458


In [99]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_inactiva_clean.insert(0, 'codi', df_poblacio_inactiva_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_inactiva_clean = df_poblacio_inactiva_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_inactiva_clean.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva
0,01,alt camp,7558,1094,2090,5614,16356
1,02,alt emporda,20492,2163,5896,27505,56056
2,03,alt penedes,16105,2289,4900,14457,37751
3,04,alt urgell,3528,477,893,4222,9120
4,05,alta ribagorca,592,102,195,569,1458


In [100]:
# Fusionar els DataFrames 'df_poblacio_inactiva_clean' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_inactiva_merged = df_poblacio_inactiva_clean.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_inactiva_merged.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva,total
0,01,alt camp,7558,1094,2090,5614,16356,45469
1,02,alt emporda,20492,2163,5896,27505,56056,143950
2,03,alt penedes,16105,2289,4900,14457,37751,110810
3,04,alt urgell,3528,477,893,4222,9120,20458
4,05,alta ribagorca,592,102,195,569,1458,3989


In [101]:
# Fusionar els DataFrames 'df_poblacio_inactiva_clean' i 'df_poblacio_genere_clean' utilitzant la columna comuna 'codi'
df_poblacio_inactiva_merged = df_poblacio_inactiva_clean.merge(df_poblacio_genere_clean[['codi', 'total']], on='codi', how='left')
df_poblacio_inactiva_merged.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva,total
0,01,alt camp,7558,1094,2090,5614,16356,45469
1,02,alt emporda,20492,2163,5896,27505,56056,143950
2,03,alt penedes,16105,2289,4900,14457,37751,110810
3,04,alt urgell,3528,477,893,4222,9120,20458
4,05,alta ribagorca,592,102,195,569,1458,3989


In [102]:
# Percentatge de població inactiva respecte al total
df_poblacio_inactiva_merged['pct_atur'] = (df_poblacio_inactiva_clean['poblacio_amb_una_altra_situacio_d\'inactivitat'] / df_poblacio_inactiva_merged['total'] * 100).round(2)

# Ràtio de població inactiva respecte al total
df_poblacio_inactiva_merged['ratio_atur'] = (df_poblacio_inactiva_clean['poblacio_amb_una_altra_situacio_d\'inactivitat'] / df_poblacio_inactiva_merged['total'])
df_poblacio_inactiva_merged.head()

,codi,comarca,"poblacio_pensionista_de_jubilacio,_prejubilacio",poblacio_pensionista_d'invalidesa,poblacio_estudiant,poblacio_amb_una_altra_situacio_d'inactivitat,total_poblacio_inactiva,total,pct_atur,ratio_atur
0,01,alt camp,7558,1094,2090,5614,16356,45469,12.35,0.123469
1,02,alt emporda,20492,2163,5896,27505,56056,143950,19.11,0.191073
2,03,alt penedes,16105,2289,4900,14457,37751,110810,13.05,0.130467
3,04,alt urgell,3528,477,893,4222,9120,20458,20.64,0.206374
4,05,alta ribagorca,592,102,195,569,1458,3989,14.26,0.142642


In [103]:
# Canviar el nom de la columna 'total' per 'total_poblacio'
df_poblacio_inactiva_merged.rename(columns={'total': 'total_poblacio'}, inplace=True)
df_poblacio_inactiva_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   codi                                             42 non-null     object 
 1   comarca                                          42 non-null     object 
 2   poblacio_pensionista_de_jubilacio,_prejubilacio  42 non-null     int64  
 3   poblacio_pensionista_d'invalidesa                42 non-null     int64  
 4   poblacio_estudiant                               42 non-null     int64  
 5   poblacio_amb_una_altra_situacio_d'inactivitat    42 non-null     int64  
 6   total_poblacio_inactiva                          42 non-null     int64  
 7   total_poblacio                                   42 non-null     int32  
 8   pct_atur                                         42 non-null     float64
 9   ratio_atur                        

In [104]:
df_poblacio_inactiva_merged['total_poblacio'].sum()

7761823

In [105]:
df_poblacio_inactiva_merged.to_csv('clean_data/2022/df_poblacio_inactiva.csv', index=False)

#### Atur per Edat<a class="anchor" id="atur"></a>

In [107]:
df_atur_edat = pd.read_csv('csv/2022/atur_edat_2022.csv', delimiter=';', skiprows=6)

In [108]:
df_atur_edat.head()

,comarca,16-24,25+,Total
0,Alt Camp,164.8,2261.8,2426.6
1,Alt Empordà,393.5,5973.8,6367.3
2,Alt Penedès,245.1,4752.7,4997.8
3,Alt Urgell,52.7,687.2,739.9
4,Alta Ribagorça,6.1,94.9,101.0


In [109]:
df_atur_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comarca  55 non-null     object 
 1   16-24    55 non-null     float64
 2   25+      55 non-null     float64
 3   Total    55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [110]:
# Netejar el DataFrame 'df_atur_edat' amb la funció 'clean_dataframe'
df_atur_edat_clean = clean_dataframe(df_atur_edat)
df_atur_edat_clean.head()

,comarca,16-24,25+,total
0,alt camp,164.8,2261.8,2426.6
1,alt emporda,393.5,5973.8,6367.3
2,alt penedes,245.1,4752.7,4997.8
3,alt urgell,52.7,687.2,739.9
4,alta ribagorca,6.1,94.9,101.0


In [111]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_edat_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [112]:
# Filtrar les comarques en el DataFrame netejat
df_atur_edat_clean = filtrar_comarques(df_atur_edat_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_edat_clean['comarca'].unique())

42

In [113]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_edat_clean.insert(0, 'codi', df_atur_edat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_edat_clean = df_atur_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,164.8,2261.8,2426.6
1,02,alt emporda,393.5,5973.8,6367.3
2,03,alt penedes,245.1,4752.7,4997.8
3,04,alt urgell,52.7,687.2,739.9
4,05,alta ribagorca,6.1,94.9,101.0


In [114]:
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   codi     42 non-null     object 
 1   comarca  42 non-null     object 
 2   16-24    42 non-null     float64
 3   25+      42 non-null     float64
 4   total    42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [115]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_edat_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_edat_clean[columnes_float] = (
    np.ceil(df_atur_edat_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   16-24    42 non-null     int32 
 3   25+      42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [116]:
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,165,2262,2427
1,02,alt emporda,394,5974,6368
2,03,alt penedes,246,4753,4998
3,04,alt urgell,53,688,740
4,05,alta ribagorca,7,95,101


In [117]:
df_atur_edat_clean.to_csv('clean_data/2022/df_atur_edat.csv', index=False)

#### Atur per Gènere<a class="anchor" id="atur_genere"></a>

In [119]:
df_atur_genere = pd.read_csv('csv/2022/atur_genere_2022.csv', delimiter=';', skiprows=6)

In [185]:
df_atur_genere.head()

,comarca,Total_homes,Total_dones,Total
0,Alt Camp,1049.4,1377.2,2426.6
1,Alt Empordà,2708.9,3658.3,6367.2
2,Alt Penedès,2049.5,2948.3,4997.8
3,Alt Urgell,306.3,433.6,739.9
4,Alta Ribagorça,46.2,54.8,101.0


In [187]:
df_atur_genere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comarca      55 non-null     object 
 1   Total_homes  55 non-null     float64
 2   Total_dones  55 non-null     float64
 3   Total        55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [189]:
# Netejar el DataFrame 'df_atur_genere' amb la funció 'clean_dataframe'
df_atur_genere_clean = clean_dataframe(df_atur_genere)
df_atur_genere_clean.head()

,comarca,total_homes,total_dones,total
0,alt camp,1049.4,1377.2,2426.6
1,alt emporda,2708.9,3658.3,6367.2
2,alt penedes,2049.5,2948.3,4997.8
3,alt urgell,306.3,433.6,739.9
4,alta ribagorca,46.2,54.8,101.0


In [191]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [193]:
# Filtrar les comarques en el DataFrame netejat
df_atur_genere_clean = filtrar_comarques(df_atur_genere_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_genere_clean['comarca'].unique())

42

In [195]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_genere_clean.insert(0, 'codi', df_atur_genere_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_genere_clean = df_atur_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1049.4,1377.2,2426.6
1,02,alt emporda,2708.9,3658.3,6367.2
2,03,alt penedes,2049.5,2948.3,4997.8
3,04,alt urgell,306.3,433.6,739.9
4,05,alta ribagorca,46.2,54.8,101.0


In [197]:
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codi         42 non-null     object 
 1   comarca      42 non-null     object 
 2   total_homes  42 non-null     float64
 3   total_dones  42 non-null     float64
 4   total        42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [199]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_genere_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_genere_clean[columnes_float] = (
    np.ceil(df_atur_genere_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   codi         42 non-null     object
 1   comarca      42 non-null     object
 2   total_homes  42 non-null     int32 
 3   total_dones  42 non-null     int32 
 4   total        42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [201]:
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1050,1378,2427
1,02,alt emporda,2709,3659,6368
2,03,alt penedes,2050,2949,4998
3,04,alt urgell,307,434,740
4,05,alta ribagorca,47,55,101


In [203]:
df_atur_genere_clean.to_csv('clean_data/2022/df_atur_genere.csv', index=False)

#### Percentatge/Ratio Estudiants<a class="anchor" id="pct_estudiants"></a>

In [131]:
df_poblacio_inactiva_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   codi                                             42 non-null     object 
 1   comarca                                          42 non-null     object 
 2   poblacio_pensionista_de_jubilacio,_prejubilacio  42 non-null     int64  
 3   poblacio_pensionista_d'invalidesa                42 non-null     int64  
 4   poblacio_estudiant                               42 non-null     int64  
 5   poblacio_amb_una_altra_situacio_d'inactivitat    42 non-null     int64  
 6   total_poblacio_inactiva                          42 non-null     int64  
 7   total_poblacio                                   42 non-null     int32  
 8   pct_atur                                         42 non-null     float64
 9   ratio_atur                        

In [132]:
# Crear un nou DataFrame amb les columnes 'codi', 'comarca', 'poblacio_estudiant' i 'total_poblacio'
df_pct_ratio_estudiants = df_poblacio_inactiva_merged[['codi', 'comarca', 'poblacio_estudiant', 'total_poblacio']].copy()

# Calcular el percentatge d'estudiants respecte al total de població
df_pct_ratio_estudiants['pct_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'] * 100).round(2)

# Calcular la ràtio d'estudiants respecte al total de població
df_pct_ratio_estudiants['ratio_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'])
df_pct_ratio_estudiants.head()

,codi,comarca,poblacio_estudiant,total_poblacio,pct_estudiants,ratio_estudiants
0,01,alt camp,2090,45469,4.60,0.045965
1,02,alt emporda,5896,143950,4.10,0.040959
2,03,alt penedes,4900,110810,4.42,0.044220
3,04,alt urgell,893,20458,4.37,0.043650
4,05,alta ribagorca,195,3989,4.89,0.048884


In [133]:
#df_pct_ratio_estudiants[['poblacio_total_16_a_24_anys', 'poblacio_homes_16_a_24_anys', 'poblacio_dones_16_a_24_anys']] = df_poblacio_genere_clean[['poblacio_total_16_a_24_anys', 'poblacio_homes_16_a_24_anys', 'poblacio_dones_16_a_24_anys']].astype(int)

In [134]:
df_pct_ratio_estudiants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   codi                42 non-null     object 
 1   comarca             42 non-null     object 
 2   poblacio_estudiant  42 non-null     int64  
 3   total_poblacio      42 non-null     int32  
 4   pct_estudiants      42 non-null     float64
 5   ratio_estudiants    42 non-null     float64
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 1.9+ KB


In [135]:
df_pct_ratio_estudiants.to_csv('clean_data/2022/df_estudiants_perc_ratio.csv', index=False)

#### Població per Nacionalitat<a class="anchor" id="poblacio_nacionalitat"></a>

In [137]:
df_poblacio_nacionalitat = pd.read_csv('csv/2022/poblacio_nacionalitat_2022.csv', delimiter=';', skiprows=7)

In [138]:
df_poblacio_nacionalitat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  55 non-null     object
 1   Espanyola                   55 non-null     int64 
 2   Resta UE                    55 non-null     int64 
 3   Resta d'Europa              55 non-null     int64 
 4   Àfrica                      55 non-null     int64 
 5   Amèrica del Nord i Central  55 non-null     int64 
 6   Amèrica del Sud             55 non-null     int64 
 7   Àsia i Oceania              55 non-null     int64 
 8   Total                       55 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 4.0+ KB


In [139]:
df_poblacio_nacionalitat.head()

,Unnamed: 0,Espanyola,Resta UE,Resta d'Europa,Àfrica,Amèrica del Nord i Central,Amèrica del Sud,Àsia i Oceania,Total
0,Alt Camp,39463,1320,491,2884,240,741,330,45469
1,Alt Empordà,108698,10998,2780,13567,2558,4222,1127,143950
2,Alt Penedès,98944,1801,792,5725,693,2022,833,110810
3,Alt Urgell,17783,944,301,457,182,668,123,20458
4,Alta Ribagorça,3473,179,62,140,37,89,9,3989


In [140]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_poblacio_nacionalitat = df_poblacio_nacionalitat.rename(columns={'Unnamed: 0': 'comarca'})
df_poblacio_nacionalitat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   comarca                     55 non-null     object
 1   Espanyola                   55 non-null     int64 
 2   Resta UE                    55 non-null     int64 
 3   Resta d'Europa              55 non-null     int64 
 4   Àfrica                      55 non-null     int64 
 5   Amèrica del Nord i Central  55 non-null     int64 
 6   Amèrica del Sud             55 non-null     int64 
 7   Àsia i Oceania              55 non-null     int64 
 8   Total                       55 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 4.0+ KB


In [141]:
# Netejar el DataFrame 'df_poblacio_nacionalitat' amb la funció 'clean_dataframe'
df_poblacio_nacionalitat_clean = clean_dataframe(df_poblacio_nacionalitat)
# Filtrar les comarques en el DataFrame netejat
df_poblacio_nacionalitat_clean = filtrar_comarques(df_poblacio_nacionalitat_clean, 'comarca')
df_poblacio_nacionalitat_clean.head()

,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,alt camp,39463,1320,491,2884,240,741,330,45469
1,alt emporda,108698,10998,2780,13567,2558,4222,1127,143950
2,alt penedes,98944,1801,792,5725,693,2022,833,110810
3,alt urgell,17783,944,301,457,182,668,123,20458
4,alta ribagorca,3473,179,62,140,37,89,9,3989


In [142]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_nacionalitat_clean.insert(0, 'codi', df_poblacio_nacionalitat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_nacionalitat_clean = df_poblacio_nacionalitat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,01,alt camp,39463,1320,491,2884,240,741,330,45469
1,02,alt emporda,108698,10998,2780,13567,2558,4222,1127,143950
2,03,alt penedes,98944,1801,792,5725,693,2022,833,110810
3,04,alt urgell,17783,944,301,457,182,668,123,20458
4,05,alta ribagorca,3473,179,62,140,37,89,9,3989


In [143]:
# Llista de nacionalitats
nacionalitats = ['espanyola', 'resta_ue', 'resta_d\'europa', 'africa', 'america_del_nord_i_central', 'america_del_sud', 'asia_i_oceania']

# Calcular el percentatge i la ràtio per a cada nacionalitat
for nacionalitat in nacionalitats:
    # Percentatge respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_pct'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total'] * 100
    ).round(2)
    
    # Ràtio respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_ratio'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total']
    )

# Mostrar les primeres files del DataFrame resultant
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total,...,resta_d'europa_pct,resta_d'europa_ratio,africa_pct,africa_ratio,america_del_nord_i_central_pct,america_del_nord_i_central_ratio,america_del_sud_pct,america_del_sud_ratio,asia_i_oceania_pct,asia_i_oceania_ratio
0,01,alt camp,39463,1320,491,2884,240,741,330,45469,...,1.08,0.010799,6.34,0.063428,0.53,0.005278,1.63,0.016297,0.73,0.007258
1,02,alt emporda,108698,10998,2780,13567,2558,4222,1127,143950,...,1.93,0.019312,9.42,0.094248,1.78,0.017770,2.93,0.029330,0.78,0.007829
2,03,alt penedes,98944,1801,792,5725,693,2022,833,110810,...,0.71,0.007147,5.17,0.051665,0.63,0.006254,1.82,0.018247,0.75,0.007517
3,04,alt urgell,17783,944,301,457,182,668,123,20458,...,1.47,0.014713,2.23,0.022338,0.89,0.008896,3.27,0.032652,0.60,0.006012
4,05,alta ribagorca,3473,179,62,140,37,89,9,3989,...,1.55,0.015543,3.51,0.035097,0.93,0.009276,2.23,0.022311,0.23,0.002256


In [144]:
df_poblacio_nacionalitat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   codi                              42 non-null     object 
 1   comarca                           42 non-null     object 
 2   espanyola                         42 non-null     int64  
 3   resta_ue                          42 non-null     int64  
 4   resta_d'europa                    42 non-null     int64  
 5   africa                            42 non-null     int64  
 6   america_del_nord_i_central        42 non-null     int64  
 7   america_del_sud                   42 non-null     int64  
 8   asia_i_oceania                    42 non-null     int64  
 9   total                             42 non-null     int64  
 10  espanyola_pct                     42 non-null     float64
 11  espanyola_ratio                   42 non-null     float64
 12  resta_ue_p

In [145]:
df_poblacio_nacionalitat_clean.to_csv('clean_data/2022/df_poblacio_nacionalitat.csv', index=False)

### Dades Socioeconòmiques<a class="anchor" id="dades_socioeconomiques"></a>

#### Repetidors<a class="anchor" id="repetidors"></a>

https://educacio.gencat.cat/ca/departament/estadistiques/estadistiques-ensenyament/cursos-anteriors/curs-2021-2022/eso/

In [149]:
df_repetidors = pd.read_csv('csv/2022/alumnes_repetidors_2022.csv', delimiter=';', encoding='latin1')

In [150]:
df_repetidors.head()

,Curs,Estudi,Codi àrea territorial,Àrea territorial,Codi comarca,Comarca,Codi municipi,Municipi,Codi districte,Naturalesa,Titularitat,Nivell,Sexe,Concert,Resultat de l'avaluació,Alumnes avaluats
0,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,1,DONA,SI,PROMOCIONEN SENSE PENDENTS,17
1,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,1,HOME,SI,PROMOCIONEN AMB PENDENTS,5
2,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,1,HOME,SI,PROMOCIONEN SENSE PENDENTS,38
3,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,2,DONA,SI,NO PROMOCIONEN,1
4,2021/2022,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PRIVAT,FUNDACIONS,2,DONA,SI,PROMOCIONEN AMB PENDENTS,6


In [151]:
df_repetidors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11679 entries, 0 to 11678
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Curs                     11679 non-null  object
 1   Estudi                   11679 non-null  object
 2   Codi àrea territorial    11679 non-null  int64 
 3   Àrea territorial         11679 non-null  object
 4   Codi comarca             11679 non-null  int64 
 5   Comarca                  11679 non-null  object
 6   Codi municipi            11679 non-null  int64 
 7   Municipi                 11679 non-null  object
 8   Codi districte           11679 non-null  int64 
 9   Naturalesa               11679 non-null  object
 10  Titularitat              11679 non-null  object
 11  Nivell                   11679 non-null  int64 
 12  Sexe                     11679 non-null  object
 13  Concert                  5073 non-null   object
 14  Resultat de l'avaluació  11679 non-nul

In [152]:
df_repetidors['Comarca'].unique()

array(['BARCELONÈS', 'ALT EMPORDÀ', 'BAIX EMPORDÀ', 'CERDANYA',
       'GARROTXA', 'GIRONÈS', "PLA DE L'ESTANY", 'RIPOLLÈS', 'SELVA',
       'ALT URGELL', 'ALTA RIBAGORÇA', 'GARRIGUES', 'NOGUERA',
       'PALLARS JUSSÀ', 'PALLARS SOBIRÀ', "PLA D'URGELL", 'SEGARRA',
       'SEGRIÀ', 'URGELL', "VAL D'ARAN", 'ALT CAMP', 'BAIX CAMP',
       'BAIX PENEDÈS', 'CONCA DE BARBERÀ', 'PRIORAT', 'TARRAGONÈS',
       'ALT PENEDÈS', 'GARRAF', 'BAIX EBRE', 'MONTSIÀ', "RIBERA D'EBRE",
       'TERRA ALTA', 'BAIX LLOBREGAT', 'VALLÈS OCCIDENTAL', 'MARESME',
       'VALLÈS ORIENTAL', 'ANOIA', 'BAGES', 'BERGUEDÀ', 'OSONA',
       'SOLSONÈS', 'MOIANÈS'], dtype=object)

In [153]:
# Netejar el DataFrame 'df_repetidors' utilitzant la funció 'clean_dataframe'
df_repetidors_clean = clean_dataframe(df_repetidors)
df_repetidors_clean.head()

,curs,estudi,codi_area_territorial,area_territorial,codi_comarca,comarca,codi_municipi,municipi,codi_districte,naturalesa,titularitat,nivell,sexe,concert,resultat_de_l'avaluacio,alumnes_avaluats
0,2021/2022,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,fundacions,1,dona,si,promocionen sense pendents,17
1,2021/2022,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,fundacions,1,home,si,promocionen amb pendents,5
2,2021/2022,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,fundacions,1,home,si,promocionen sense pendents,38
3,2021/2022,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,fundacions,2,dona,si,no promocionen,1
4,2021/2022,educacio secundaria obligatoria,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,privat,fundacions,2,dona,si,promocionen amb pendents,6


In [154]:
# Filtrar les dades del DataFrame per obtenir només les comarques rellevants
df_repetidors_clean = filtrar_comarques(df_repetidors_clean, 'comarca')

In [155]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
display(df_repetidors_clean['comarca'].unique())
print(len(df_repetidors_clean['comarca'].unique()))

array(['barcelones', 'alt emporda', 'baix emporda', 'cerdanya',
       'garrotxa', 'girones', "pla de l'estany", 'ripolles', 'selva',
       'alt urgell', 'alta ribagorca', 'garrigues', 'noguera',
       'pallars jussa', 'pallars sobira', "pla d'urgell", 'segarra',
       'segria', 'urgell', 'alt camp', 'baix camp', 'baix penedes',
       'conca de barbera', 'priorat', 'tarragones', 'alt penedes',
       'garraf', 'baix ebre', 'montsia', "ribera d'ebre", 'terra alta',
       'baix llobregat', 'valles occidental', 'maresme',
       'valles oriental', 'anoia', 'bages', 'bergueda', 'osona',
       'solsones', 'moianes'], dtype=object)

41


In [156]:
# Obtenir els valors únics de la columna 'resultat_de_l'avaluacio' en el DataFrame netejat
df_repetidors_clean['resultat_de_l\'avaluacio'].unique()

array(['promocionen sense pendents', 'promocionen amb pendents',
       'no promocionen'], dtype=object)

In [157]:
# Filtrar el DataFrame per mantenir només les files amb 'resultat_de_l'avaluacio' igual a 'no promocionen'
df_no_promocionen = df_repetidors_clean[df_repetidors_clean['resultat_de_l\'avaluacio'] == 'no promocionen']

# Agrupar les dades per 'comarca' i 'sexe', sumant els valors de 'alumnes_avaluats'
df_repetidors = df_no_promocionen.groupby(['comarca', 'sexe'])['alumnes_avaluats'].sum().reset_index()

# Renombrar la columna 'alumnes_avaluats' a 'repetixen'
df_repetidors = df_repetidors.rename(columns={'alumnes_avaluats': 'repetixen'})

# Mostrar les primeres files del DataFrame resultant
df_repetidors.head()

,comarca,sexe,repetixen
0,alt camp,dona,42
1,alt camp,home,83
2,alt emporda,dona,123
3,alt emporda,home,216
4,alt penedes,dona,76


In [158]:
# Afegim la columna 'codi'  per fer match amb els noms de les comarques
df_repetidors.insert(0, 'codi', df_repetidors['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_repetidors = df_repetidors.sort_values(by='codi').reset_index(drop=True)
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,42
1,01,alt camp,home,83
2,02,alt emporda,dona,123
3,02,alt emporda,home,216
4,03,alt penedes,dona,76


In [159]:
df_repetidors.to_csv('clean_data/2022/df_repetidors.csv', index=False)

### Total Repetidors per Comarca<a class="anchor" id="repetidors_comarca"></a>

In [161]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,42
1,01,alt camp,home,83
2,02,alt emporda,dona,123
3,02,alt emporda,home,216
4,03,alt penedes,dona,76


In [162]:
# Agrupar les dades per 'codi' i 'comarca', sumant els valors de 'repetixen'
df_total_repetidors = df_repetidors.groupby(['codi', 'comarca'])['repetixen'].sum().reset_index()

# Renombrar la columna 'repetixen' a 'total_repetixen'
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen'}, inplace=True)
df_total_repetidors.head()

,codi,comarca,total_repetixen
0,01,alt camp,125
1,02,alt emporda,339
2,03,alt penedes,241
3,04,alt urgell,50
4,05,alta ribagorca,13


In [163]:
df_pct_ratio_estudiants.head()

,codi,comarca,poblacio_estudiant,total_poblacio,pct_estudiants,ratio_estudiants
0,01,alt camp,2090,45469,4.60,0.045965
1,02,alt emporda,5896,143950,4.10,0.040959
2,03,alt penedes,4900,110810,4.42,0.044220
3,04,alt urgell,893,20458,4.37,0.043650
4,05,alta ribagorca,195,3989,4.89,0.048884


In [164]:
# Fusionar els DataFrames 'df_total_repetidors' i 'df_pct_ratio_estudiants' utilitzant la columna comuna 'codi'
df_total_repetidors_merged = df_total_repetidors.merge(df_pct_ratio_estudiants[['codi', 'poblacio_estudiant']], on='codi', how='left')

# Seleccionar només les columnes necessàries del DataFrame resultant
df_total_repetidors_merged = df_total_repetidors_merged[['codi', 'comarca', 'total_repetixen', 'poblacio_estudiant']].copy()
df_total_repetidors_merged.head()

,codi,comarca,total_repetixen,poblacio_estudiant
0,01,alt camp,125,2090
1,02,alt emporda,339,5896
2,03,alt penedes,241,4900
3,04,alt urgell,50,893
4,05,alta ribagorca,13,195


In [165]:
# Calcular el percentatge de repetidors respecte a la població estudiant
df_total_repetidors_merged['pct_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'] * 100).round(2)

# Calcular la ràtio de repetidors respecte a la població estudiant
df_total_repetidors_merged['ratio_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'])
df_total_repetidors_merged.head()

,codi,comarca,total_repetixen,poblacio_estudiant,pct_repetidors,ratio_repetidors
0,01,alt camp,125,2090,5.98,0.059809
1,02,alt emporda,339,5896,5.75,0.057497
2,03,alt penedes,241,4900,4.92,0.049184
3,04,alt urgell,50,893,5.60,0.055991
4,05,alta ribagorca,13,195,6.67,0.066667


In [166]:
df_total_repetidors_merged.to_csv('clean_data/2022/df_total_repetidors.csv', index=False)

### Repetidors Població<a class="anchor" id="repetidors_poblacio"></a>

In [168]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,42
1,01,alt camp,home,83
2,02,alt emporda,dona,123
3,02,alt emporda,home,216
4,03,alt penedes,dona,76


In [169]:
df_total_repetidors_merged.head()

,codi,comarca,total_repetixen,poblacio_estudiant,pct_repetidors,ratio_repetidors
0,01,alt camp,125,2090,5.98,0.059809
1,02,alt emporda,339,5896,5.75,0.057497
2,03,alt penedes,241,4900,4.92,0.049184
3,04,alt urgell,50,893,5.60,0.055991
4,05,alta ribagorca,13,195,6.67,0.066667


In [170]:
# Fusionar 'df_repetidors' amb 'df_total_repetidors_merged' per afegir 'comarca' i 'poblacio_estudiant'
df_repetidors_genere_merged = df_repetidors.merge(
    df_total_repetidors_merged[['codi', 'comarca', 'poblacio_estudiant']], 
    on='codi', 
    how='left'
)
df_repetidors_genere_merged.head()

,codi,comarca_x,sexe,repetixen,comarca_y,poblacio_estudiant
0,01,alt camp,dona,42,alt camp,2090
1,01,alt camp,home,83,alt camp,2090
2,02,alt emporda,dona,123,alt emporda,5896
3,02,alt emporda,home,216,alt emporda,5896
4,03,alt penedes,dona,76,alt penedes,4900


In [171]:
# Eliminar 'comarca_y' i renombrar 'comarca_x' a 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.drop(columns=['comarca_y']).rename(columns={'comarca_x': 'comarca'})
df_repetidors_genere_merged.head()

,codi,comarca,sexe,repetixen,poblacio_estudiant
0,01,alt camp,dona,42,2090
1,01,alt camp,home,83,2090
2,02,alt emporda,dona,123,5896
3,02,alt emporda,home,216,5896
4,03,alt penedes,dona,76,4900


In [172]:
# Crear df_dones (mujeres) i df_homes (hombres) separats per genere
df_dones = df_repetidors_genere_merged[df_repetidors_genere_merged['sexe'] == 'dona']
df_homes = df_repetidors_genere_merged[df_repetidors_genere_merged['sexe'] == 'home']

display(df_dones.head())
display(df_homes.head())

,codi,comarca,sexe,repetixen,poblacio_estudiant
0,01,alt camp,dona,42,2090
2,02,alt emporda,dona,123,5896
4,03,alt penedes,dona,76,4900
6,04,alt urgell,dona,15,893
8,05,alta ribagorca,dona,7,195


,codi,comarca,sexe,repetixen,poblacio_estudiant
1,01,alt camp,home,83,2090
3,02,alt emporda,home,216,5896
5,03,alt penedes,home,165,4900
7,04,alt urgell,home,35,893
9,05,alta ribagorca,home,6,195


In [173]:
# Calcular el percentatge de repetidors per dones (mujeres)
df_dones = df_dones.copy()
df_homes = df_homes.copy()
df_dones.loc[:, 'pct_repetidors_dones'] = (df_dones['repetixen'] / df_dones['poblacio_estudiant']) * 100
df_dones.loc[:, 'ratio_repetidors_dones'] = df_dones['repetixen'] / df_dones['poblacio_estudiant']

# Calcular el percentatge de repetidors per homes (hombres)
df_homes.loc[:, 'pct_repetidors_homes'] = (df_homes['repetixen'] / df_homes['poblacio_estudiant']) * 100
df_homes.loc[:, 'ratio_repetidors_homes'] = df_homes['repetixen'] / df_homes['poblacio_estudiant']

# Unir els resultats de dones i homes en un sol DataFrame
df_repetidors_poblacio = pd.concat([df_dones, df_homes])

# Agrupar per 'codi' i 'comarca' per calcular els totals per comarca
df_repetidors_poblacio = df_repetidors_poblacio.groupby(['codi', 'comarca']).agg({
    'repetixen': 'sum', 
    'pct_repetidors_dones': 'mean',
    'ratio_repetidors_dones': 'mean', 
    'pct_repetidors_homes': 'mean', 
    'ratio_repetidors_homes': 'mean' 
}).reset_index()

df_repetidors_poblacio.head()

,codi,comarca,repetixen,pct_repetidors_dones,ratio_repetidors_dones,pct_repetidors_homes,ratio_repetidors_homes
0,01,alt camp,125,2.009569,0.020096,3.971292,0.039713
1,02,alt emporda,339,2.086160,0.020862,3.663501,0.036635
2,03,alt penedes,241,1.551020,0.015510,3.367347,0.033673
3,04,alt urgell,50,1.679731,0.016797,3.919373,0.039194
4,05,alta ribagorca,13,3.589744,0.035897,3.076923,0.030769


In [174]:
# Renombrar la columna 'repetixen' a 'total_repetixen'
df_repetidors_poblacio = df_repetidors_poblacio.rename(columns={'repetixen': 'total_repetixen'})

df_repetidors_poblacio.head()

,codi,comarca,total_repetixen,pct_repetidors_dones,ratio_repetidors_dones,pct_repetidors_homes,ratio_repetidors_homes
0,01,alt camp,125,2.009569,0.020096,3.971292,0.039713
1,02,alt emporda,339,2.086160,0.020862,3.663501,0.036635
2,03,alt penedes,241,1.551020,0.015510,3.367347,0.033673
3,04,alt urgell,50,1.679731,0.016797,3.919373,0.039194
4,05,alta ribagorca,13,3.589744,0.035897,3.076923,0.030769


In [175]:
df_repetidors_poblacio.to_csv('clean_data/2022/df_repetidors_poblacio.csv', index=False)